In [1]:
cd gcn

D:\Downloads\Study\Graduation Research\GCN\gcn\gcn


In [3]:
from __future__ import division
from __future__ import print_function

import time

import tensorflow as tf

from gcn.utils import *
from gcn.models import GCN, MLP


In [4]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)


In [5]:
# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')


In [6]:
# Load data

"""
    Loads input data from gcn/data directory

    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data). 
             Adjacent matrix, feature matrix, label matrix of each phase with corresponding mask
"""

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)


D:\Downloads\Study\Graduation Research\GCN\gcn\venv\lib\site-packages\scipy\sparse\lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
D:\Downloads\Study\Graduation Research\GCN\gcn\venv\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


In [ ]:
# Some preprocessing
features = preprocess_features(features)
if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
elif FLAGS.model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))


(array([[   0, 1274],
        [   0, 1247],
        [   0, 1194],
        ...,
        [2707,  329],
        [2707,  186],
        [2707,   19]], dtype=int32),
 array([0.11111111, 0.11111111, 0.11111111, ..., 0.07692308, 0.07692308,
        0.07692308], dtype=float32),
 (2708, 1433))

In [9]:
features


(array([[   0, 1274],
        [   0, 1247],
        [   0, 1194],
        ...,
        [2707,  329],
        [2707,  186],
        [2707,   19]], dtype=int32),
 array([0.11111111, 0.11111111, 0.11111111, ..., 0.07692308, 0.07692308,
        0.07692308], dtype=float32),
 (2708, 1433))

In [7]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}


In [8]:
# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation for early stopping
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Instructions for updating:
Use tf.cast instead.


Epoch: 0001 train_loss= 1.95461 train_acc= 0.07857 val_loss= 1.95068 val_acc= 0.17600 time= 2.79149
Epoch: 0002 train_loss= 1.94861 train_acc= 0.25714 val_loss= 1.94722 val_acc= 0.32400 time= 0.12093


...

Epoch: 0191 train_loss= 0.62657 train_acc= 0.95714 val_loss= 1.05898 val_acc= 0.78600 time= 0.11893
Epoch: 0192 train_loss= 0.64796 train_acc= 0.95000 val_loss= 1.05727 val_acc= 0.78800 time= 0.10594


Epoch: 0193 train_loss= 0.62399 train_acc= 0.97857 val_loss= 1.05572 val_acc= 0.78800 time= 0.10370
Epoch: 0194 train_loss= 0.61233 train_acc= 0.98571 val_loss= 1.05403 val_acc= 0.78600 time= 0.10794


Epoch: 0195 train_loss= 0.59135 train_acc= 0.97857 val_loss= 1.05217 val_acc= 0.78600 time= 0.13192
Epoch: 0196 train_loss= 0.64115 train_acc= 0.93571 val_loss= 1.05070 val_acc= 0.78800 time= 0.12892


Epoch: 0197 train_loss= 0.58866 train_acc= 0.95714 val_loss= 1.04922 val_acc= 0.78600 time= 0.14992
Epoch: 0198 train_loss= 0.59634 train_acc= 0.95714 val_loss= 1.04746 val_acc= 0.78600 time= 0.11693


Epoch: 0199 train_loss= 0.58434 train_acc= 0.98571 val_loss= 1.04561 val_acc= 0.78800 time= 0.09894
Epoch: 0200 train_loss= 0.61550 train_acc= 0.95714 val_loss= 1.04385 val_acc= 0.79000 time= 0.10294
Optimization Finished!


In [9]:
feed_dict_val = construct_feed_dict(features, support, y_test, test_mask, placeholders)
# Get the prediction of the test set
out = sess.run([model.predict()], feed_dict=feed_dict_val)
# First node prediction
out[0][1708]


array([0.13206233, 0.20007877, 0.17057942, 0.14500543, 0.08709805,
       0.14569305, 0.11948295], dtype=float32)

In [10]:
# Compare real labels and predict labels 
print('Indies' + '\t' + 'Real labels' + '\t' + 'Predict labels')
for i in range(1708, 2708):
    print(str(i) + '\t' + str(np.argmax(y_test[i])) + '\t' + str(np.argmax(out[0][i])))


Indies	Real labels	Predict labels
1708	3	1
1709	2	2
1710	2	2
1711	2	2
1712	2	2
1713	0	0
1714	2	2
1715	2	2
1716	2	2
1717	2	2
1718	2	2
1719	2	2
1720	2	2
1721	2	2
1722	2	2
1723	2	2
1724	2	2
1725	2	2
1726	2	2
1727	2	2
1728	3	3
1729	2	2
1730	2	2
1731	2	2
1732	2	2
1733	2	1
1734	2	2
1735	1	2
1736	2	2
1737	2	2
1738	2	2
1739	2	2
1740	2	2
1741	3	0
1742	2	2
1743	2	2
1744	2	2
1745	2	2
1746	2	2
1747	2	2
1748	2	2
1749	2	2
1750	2	2
1751	2	2
1752	2	2
1753	2	2
1754	2	2
1755	2	2
1756	2	2
1757	2	2
1758	2	2
1759	2	2
1760	2	2
1761	2	2
1762	2	2
1763	2	2
1764	5	2
1765	2	2
1766	2	2
1767	1	1
1768	1	1
1769	1	1
1770	1	1
1771	1	1
1772	1	1
1773	1	1
1774	4	1
1775	1	1
1776	1	1
1777	1	1
1778	1	1
1779	1	1
1780	1	1
1781	1	1
1782	1	1
1783	1	1
1784	1	1
1785	4	4
1786	1	1
1787	1	1
1788	1	1
1789	1	1
1790	1	1
1791	1	1
1792	3	6
1793	4	5
1794	4	4
1795	4	4
1796	4	4
1797	1	1
1798	1	1
1799	3	3
1800	1	1
1801	0	1
1802	3	1
1803	0	6
1804	2	2
1805	1	1
1806	3	3
1807	3	3
1808	3	3
1809	3	3
1810	3	3
1811	3	3
1812	3	3
1813	3	3
1814	3	3
181

2550	3	3
2551	3	3
2552	3	3
2553	5	6
2554	2	2
2555	3	3
2556	2	2
2557	2	2
2558	2	2
2559	3	3
2560	2	2
2561	2	0
2562	0	6
2563	4	4
2564	4	4
2565	3	4
2566	3	4
2567	3	4
2568	3	3
2569	3	2
2570	3	3
2571	3	2
2572	3	3
2573	3	3
2574	3	3
2575	0	0
2576	0	0
2577	3	3
2578	0	3
2579	3	3
2580	0	3
2581	2	2
2582	3	3
2583	4	4
2584	1	2
2585	2	2
2586	5	3
2587	4	4
2588	3	3
2589	3	3
2590	3	4
2591	1	1
2592	5	5
2593	3	3
2594	4	4
2595	3	3
2596	2	2
2597	2	2
2598	1	1
2599	3	3
2600	3	2
2601	3	3
2602	3	3
2603	3	3
2604	6	6
2605	3	3
2606	3	2
2607	3	2
2608	6	6
2609	3	3
2610	3	3
2611	3	0
2612	2	2
2613	3	3
2614	2	2
2615	4	3
2616	2	2
2617	4	5
2618	2	2
2619	2	2
2620	1	0
2621	5	5
2622	6	6
2623	4	4
2624	3	3
2625	3	3
2626	3	3
2627	2	2
2628	5	5
2629	3	3
2630	3	3
2631	4	4
2632	3	3
2633	3	3
2634	3	3
2635	3	3
2636	3	3
2637	4	4
2638	6	6
2639	0	6
2640	3	5
2641	2	2
2642	2	2
2643	2	2
2644	5	0
2645	4	4
2646	4	4
2647	4	4
2648	4	4
2649	6	6
2650	3	3
2651	2	2
2652	2	2
2653	0	0
2654	2	6
2655	2	3
2656	2	6
2657	2	2
2658	2	2
2659	3	3
2660	4	0
2

In [11]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


Test set results: cost= 1.00631 accuracy= 0.80900 time= 0.05297
